<a href="https://colab.research.google.com/github/remidion/PGM-Project/blob/main/Timeliness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/remidion/PGM-Project/blob/main/main.ipynb)

In [ ]:
# Imports
from getpass import getpass
import pandas as pd
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
if os.path.exists('PGM-Project'):
  !git -C PGM-Project/ pull
else:
  u = input('GitHub username: ')
  p = getpass('GitHub password: ')
  !git clone https://$u:$p@github.com/remidion/PGM-Project.git
  del u, p

Already up to date.


# Load Data

In [ ]:
out_df = pd.read_csv("PGM-Project/data/route_01_full.csv", index_col=[0,1], header=[0,1])
out_df

scheduled           ... timeliness         
stop_sequence                          1.0      2.0  ...        8.0      9.0
service_date route_start_time                        ...                    
2018-08-02   1900-01-01 05:10:00   18600.0  18780.0  ...       Late     Late
             1900-01-01 05:30:00   19800.0  19980.0  ...       Late     Late
             1900-01-01 05:50:00   21000.0  21180.0  ...       Late     Late
             1900-01-01 06:06:00   21960.0  22140.0  ...       Late     Late
             1900-01-01 06:21:00   22860.0  23040.0  ...    On Time  On Time
...                                    ...      ...  ...        ...      ...
2019-09-30   1900-01-01 23:40:00   85200.0  85380.0  ...    On Time  On Time
             1900-01-01 23:54:00   86040.0  86220.0  ...    On Time  On Time
             1900-01-02 00:22:00   87720.0  87900.0  ...      Early    Early
             1900-01-02 00:37:00   88620.0  88800.0  ...    On Time    Early
             1900-01-02 00:52:00   89520.0  89700.0  ...    On Time    Early

[37203 rows x 18 columns]

In [ ]:
out2_df = pd.read_csv("PGM-Project/data/route_01_binned.csv", index_col=[0], header=[0,1])
out2_df

service_date scheduled  ... timeliness            route_bin
stop_sequence Unnamed: 1_level_1       1.0  ...        9.0  Unnamed: 20_level_1
0                     2018-08-03   21960.0  ...    On Time  1900-01-01 06:00:00
1                     2018-08-03   23700.0  ...       Late  1900-01-01 06:30:00
2                     2018-08-03   25560.0  ...       Late  1900-01-01 07:00:00
3                     2018-08-03   27300.0  ...      Early  1900-01-01 07:30:00
4                     2018-08-03   28980.0  ...       Late  1900-01-01 08:00:00
...                          ...       ...  ...        ...                  ...
9153                  2019-09-30   81240.0  ...    On Time  1900-01-01 22:30:00
9154                  2019-09-30   83580.0  ...      Early  1900-01-01 23:00:00
9155                  2019-09-30   85200.0  ...    On Time  1900-01-01 23:30:00
9156                  2019-09-30   87720.0  ...      Early  1900-01-02 00:00:00
9157                  2019-09-30   88620.0  ...      Early  1900-01-02 00:30:00

[9158 rows x 20 columns]

In [ ]:
n_bins = int(out2_df["route_bin"].nunique())

# Training

In [ ]:
X = out_df['scheduled'].to_numpy()
y = out_df['timeliness'].to_numpy()

X2 = out2_df['scheduled'].to_numpy()
y2 = out2_df['timeliness'].to_numpy()

In [ ]:
X

array([[18600., 18780., 18960., ..., 19680., 19920., 19980.],
       [19800., 19980., 20160., ..., 20880., 21120., 21180.],
       [21000., 21180., 21360., ..., 22080., 22320., 22380.],
       ...,
       [87720., 87900., 88080., ..., 89040., 89280., 89520.],
       [88620., 88800., 88980., ..., 89940., 90180., 90420.],
       [89520., 89700., 89880., ..., 90840., 91080., 91320.]])

In [ ]:
y

array([['On Time', 'On Time', 'On Time', ..., 'Late', 'Late', 'Late'],
       ['On Time', 'On Time', 'On Time', ..., 'Late', 'Late', 'Late'],
       ['On Time', 'On Time', 'On Time', ..., 'Late', 'Late', 'Late'],
       ...,
       ['Late', 'Late', 'Late', ..., 'On Time', 'Early', 'Early'],
       ['On Time', 'On Time', 'On Time', ..., 'On Time', 'On Time',
        'Early'],
       ['On Time', 'On Time', 'On Time', ..., 'Early', 'On Time',
        'Early']], dtype=object)

## Aligned Bins

In [ ]:
# Split dataset into training set and test set
test_fraction = 0.2
split_idx = int(X.shape[0] * test_fraction)
X_train, X_test, y_train, y_test = X[:-split_idx], X[-split_idx:], y[:-split_idx], y[-split_idx:]

In [ ]:
# Reshape into (Days x time bins x stops)
shape = (X2.shape[0]//n_bins, n_bins, X2.shape[1])

X2 = X2.reshape(shape) 
y2 = y2.reshape(shape) 

In [ ]:
# Split dataset #2 into training set and test set
test_fraction = 0.2
split_idx = int(X2.shape[0] * test_fraction)
X2_train, X2_test, y2_train, y2_test = X2[:-split_idx], X2[-split_idx:], y2[:-split_idx], y2[-split_idx:]

In [ ]:
X2_train.shape

(193, 38, 9)

In [ ]:
X2_test.shape

(48, 38, 9)

## Mode Baseline

In [ ]:
from collections import Counter

def baseline(df, verbose=False):
  c = Counter(df.flatten())
  counts = np.array(list(c.values()))
  if verbose: print('Values: ', dict(c.items()))
  ratio = counts/sum(counts)
  mode_performance = max(ratio)
  if verbose: print('Distribution: ', ratio)
  return mode_performance

def b2(df, verbose=True):
  c = Counter(df.flatten())
  myDict = dict(c.items())
  if verbose: print(myDict)
  s = sum(c.values())
  key = max(myDict, key=myDict.get)
  baseline = f'{myDict[key]/s} for "{key}"'
  #print(key, myDict[key]/s)
  return baseline

#print(' y:', b2(y))
#print()
#print('y2:', b2(y2))
#print()

print('y2:', b2(y2_train))

{'Early': 11368, 'On Time': 31270, 'Late': 23368}
y2: 0.4737448110777808 for "On Time"


In [ ]:
print(X_train.shape,X2_train.shape, X2_train.reshape(-1,9).shape)
print(y_train.shape,y2_train.shape, y2_train.reshape(-1,9).shape)

(29763, 9) (193, 38, 9) (7334, 9)
(29763, 9) (193, 38, 9) (7334, 9)


# Inference

## Markov Chain - Stops id

In [ ]:
import math

def get_dtw(knots, x, homogenous=False):
  if homogeneous: return 0
  for k in range(len(knots) - 1):
        if x >= knots[k] and x < knots[k + 1]:
            return k


def parametrization(state_dict, X_tr, y_tr, knots = None, homogeneous = True):
      
    N, T = X_tr.shape

    # Initialize the Transition Matrix.ces
    K = 1 if homogeneous else len(knots)-1
    transition_matrix = np.zeros((K,3,3))
    pi = np.zeros((K, 3))

    for i in range(N):

      # Ground Truth for first stop of sequence
      actual_state = state_dict[y_tr[i,0]] 

      dtw = get_dtw(knots, X_tr[i,0], homogeneous)
      pi[dtw, actual_state] += 1

      for j in range(T - 1):
        actual_state = state_dict[y_tr[i,j]]
        next_state = state_dict[y_tr[i,j + 1]]
        
        dtw = get_dtw(knots, X_tr[i,j], homogeneous)
        transition_matrix[dtw, actual_state, next_state] +=  1

    for k in range(K):

      if sum(pi[k,:]) == 0:
        pi[k,:] = [0,1,0]  # Baseline is On Time
        #pi[k,:] = [1/3, 1/3, 1/3]
      else:
        pi[k,:] = pi[k,:] / sum(pi[k,:])
      
      #Check first divison by 0
      #if transition_matrix[dtw,:,:].sum(axis = 1)[0] == 0 and transition_matrix[dtw,:,:].sum(axis = 1)[1] == 0 and transition_matrix[dtw,:,:].sum(axis = 1)[2] == 0:
      #    transition_matrix[dtw,:,:] =  transition_matrix[dtw - 1,:,:]
      #else:
      if transition_matrix[k,:,:].sum(axis = 1)[0] == 0:
        transition_matrix[k,0,:] = [1/3, 1/3, 1/3]
      if transition_matrix[k,:,:].sum(axis = 1)[1] == 0:
        transition_matrix[k,1,:] = [1/3, 1/3, 1/3]
      if transition_matrix[k,:,:].sum(axis = 1)[2] == 0:
        transition_matrix[k,2,:] = [1/3, 1/3, 1/3]
          
      transition_matrix[k,:,:] = (transition_matrix[k,:,:].T/transition_matrix[k,:,:].sum(axis = 1)).T

    return pi, transition_matrix

def pred_next_trip_delays(state_dict, pi, transition_matrix, X_tst, knots = None, homogeneous = True):

    future_states = []

    dtw = get_dtw(knots, X_tst[0], homogeneous)
    first_stop_state = np.random.choice(list(state_dict.keys()), p = pi[dtw, :])

    future_states.append(first_stop_state)

    for i in range(len(X_tst)):
      current_state = state_dict[future_states[-1]]

      dtw = get_dtw(knots, X_tst[i], homogeneous)
      next_state = np.random.choice(list(state_dict.keys()), p = transition_matrix[dtw, current_state, :])
      future_states.append(next_state)

    return future_states

def pred_next_stop_delay(current_state, state_dict, transition_matrix, X_tst, knots, homogeneous = True):

    dtw = get_dtw(knots, X_tst, homogeneous)
    return np.random.choice(list(state_dict.keys()), p = transition_matrix[dtw, state_dict[current_state], :])



In [ ]:
#----- Parameters ----- 
slidewindow_len = 60 #Number of previous days to train on
homogeneous = True
knots = [0, 21600, 32400, 50400, 64800, 93600] #DTWs time limits (for inhomogeneous chains)
#----------------------

#Init
state_dict = {'Early': 0, 'On Time': 1, 'Late': 2}
accu1 = 0
accu2 = 0

#Usefull constants
N, T = X2_train.shape
for h in [True, False]:
  homogeneous=h
  print('homogenous =',h)
  print()

  for k in range(slidewindow_len, N):

      X_tr = X2_train[k-slidewindow_len:k,:]
      y_tr = y2_train[k-slidewindow_len:k,:]
      X_tst = X2_train[k,:]
      y_tst = y2_train[k,:]
      pi, transition_matrix = parametrization(state_dict, X_tr, y_tr, knots, homogeneous)


      #Predict the next trip sequence of states
      future_states = pred_next_trip_delays(state_dict, pi, transition_matrix, X_tst, knots, homogeneous)

      for t in range(T):
        if y_tst[t] == future_states[t]:
            accu1 += 1
      

      #Predict the state of the next stop
      dtw = get_dtw(knots, X_tst[0], homogeneous)
      first_state = np.random.choice(list(state_dict.keys()), p = pi[dtw, :]) #Prediction of the first stop state is random
      
      if y_tst[0] == first_state:
          accu2 += 1

      for t in range(1, T):
          next_state = pred_next_stop_delay(y_tst[t-1], state_dict, transition_matrix, X_tst[t], knots, homogeneous)
          
          if y_tst[t] == next_state:
              accu2 += 1


  accu1 = accu1 /((N - slidewindow_len) * T)
  accu2 = accu2 /((N - slidewindow_len) * T)
  print('Sequence:', accu1)
  print('Next', accu2)


homogenous = True



ValueError: ignored

accu1 = Estimate whole sequence \\
accu2 = Estimate state for next stop ID \\
 \\
homogeneous:

*   accu1: 0.43205470637966764
*   accu2: 0.6853260910226772

inhomogeneous (DTW = 1 hour):

*   accu1 0.4616673774306257
*   accu2 0.6461181065248132

inhomogeneous (DTW = peak vs off-peak):

*   accu1 0.44364015891185776
*   accu2 0.6807622382331157



In [ ]:
# Split dataset #2 into training set and validation set
valid_fraction = 0.2
split_idx = int(X2_train.shape[0] * valid_fraction)
X2_tr, X2_val, y2_tr, y2_val = X2[:-split_idx], X2_train[-split_idx:], y2_train[:-split_idx], y2_train[-split_idx:]
print(X2_tr.shape, X2_val.shape, y2_tr.shape, y2_val.shape)

(203, 38, 9) (38, 38, 9) (155, 38, 9) (38, 38, 9)


In [ ]:
#----- Parameters ----- 
homogeneous = True
knots = [0, 21600, 32400, 50400, 64800, 93600] #DTWs time limits (for inhomogeneous chains)
#----------------------

#Init
state_dict = {'Early': 0, 'On Time': 1, 'Late': 2}
accu1 = 0
accu2 = 0

#Usefull constants
N, T = X2_tr.shape
for h in [True, False]:
  homogeneous=h
  print('homogenous =',h)
  print()

  # Train

  # Evaluate
  for x in X2_tr:

      X_tr = X_train[i-slidewindow_len:i,:]
      y_tr = y_train[i-slidewindow_len:i,:]
      X_tst = X_train[i,:]
      y_tst = y_train[i,:]
      pi, transition_matrix = parametrization(state_dict, X_tr, y_tr, knots, homogeneous)


      #Predict the next trip sequence of states
      future_states = pred_next_trip_delays(state_dict, pi, transition_matrix, X_tst, knots, homogeneous)

      for t in range(T):
        if y_tst[t] == future_states[t]:
            accu1 += 1
      

      #Predict the state of the next stop
      dtw = get_dtw(knots, X_tst[0], homogeneous)
      first_state = np.random.choice(list(state_dict.keys()), p = pi[dtw, :]) #Prediction of the first stop state is random
      
      if y_tst[0] == first_state:
          accu2 += 1

      for t in range(1, T):
          next_state = pred_next_stop_delay(y_tst[t-1], state_dict, transition_matrix, X_tst[t], knots, homogeneous)
          
          if y_tst[t] == next_state:
              accu2 += 1


  accu1 = accu1 /((N - slidewindow_len) * T)
  accu2 = accu2 /((N - slidewindow_len) * T)
  print('Sequence:', accu1)
  print('Next', accu2)


In [ ]:
import math

def parametrization(state_dict, X_tr, y_tr, knots, time_homogeneous = True, stop_homogeneous = True):
      
    N, T = X_tr.shape

    if not time_homogeneous:
        transition_matrix = np.zeros((len(knots) - 1,3,3))
        pi = np.zeros((len(knots) - 1, 3))

    elif not stop_homogeneous:
        transition_matrix = np.zeros((T - 1,3,3))
        pi = np.zeros(3)
    
    else:
        transition_matrix = np.zeros((3,3))
        pi = np.zeros(3)

    for i in range(N):

        actual_state = state_dict[y_tr[i,0]] 

        if not time_homogeneous:
            for k in range(len(knots) - 1):
                if (X_tr[i,0] >= knots[k]) and (X_tr[i,0] < knots[k + 1]):
                    dtw = k
            pi[dtw, actual_state] += 1
  
        else:
            pi[actual_state] += 1

        for j in range(T - 1):
            actual_state = state_dict[y_tr[i,j]]
            next_state = state_dict[y_tr[i,j + 1]]

            if not time_homogeneous:
                for k in range(len(knots) - 1):
                    if X_tr[i,j] >= knots[k] and X_tr[i,j]< knots[k + 1]:
                        dtw = k
                transition_matrix[dtw, actual_state, next_state] +=  1
            
            elif not stop_homogeneous:
                transition_matrix[j, actual_state, next_state] +=  1

            else:
                transition_matrix[actual_state, next_state] +=  1

    #Normalization
    if not time_homogeneous:

        for k in range(0, len(knots) - 1):

            if sum(pi[k,:]) == 0:
                pi[k,:] = [1/3, 1/3, 1/3]
            else:
                pi[k,:] = pi[k,:] / sum(pi[k,:])
            
            if transition_matrix[k,:,:].sum(axis = 1)[0] == 0:
                transition_matrix[k,0,:] = [1/3, 1/3, 1/3]
            if transition_matrix[k,:,:].sum(axis = 1)[1] == 0:
                transition_matrix[k,1,:] = [1/3, 1/3, 1/3]
            if transition_matrix[k,:,:].sum(axis = 1)[2] == 0:
                transition_matrix[k,2,:] = [1/3, 1/3, 1/3]
                
            transition_matrix[k,:,:] = (transition_matrix[k,:,:].T/transition_matrix[k,:,:].sum(axis = 1)).T

    elif not stop_homogeneous:

        for j in range(T - 1):
            pi = pi/sum(pi)
            
            if transition_matrix[j,:,:].sum(axis = 1)[0] == 0:
                transition_matrix[j,0,:] = [1/3, 1/3, 1/3]
            if transition_matrix[j,:,:].sum(axis = 1)[1] == 0:
                transition_matrix[j,1,:] = [1/3, 1/3, 1/3]
            if transition_matrix[j,:,:].sum(axis = 1)[2] == 0:
                transition_matrix[j,2,:] = [1/3, 1/3, 1/3]
                
            transition_matrix[j,:,:] = (transition_matrix[j,:,:].T/transition_matrix[j,:,:].sum(axis = 1)).T

    else:      
        pi = pi/sum(pi)
        
        #Check first divison by 0
        if transition_matrix.sum(axis = 1)[0] == 0 or transition_matrix.sum(axis = 1)[1] == 0 or transition_matrix.sum(axis = 1)[2] == 0:
            print('error: one or more states not seen in training set. Increase slidewindow_len.')
        else:
            transition_matrix = (transition_matrix.T/transition_matrix.sum(axis = 1)).T

    return pi, transition_matrix

def pred_next_trip_delays(state_dict, pi, transition_matrix, X_tst, knots, time_homogeneous = True, stop_homogeneous = True):

    future_states = []

    if not time_homogeneous:

        for k in range(len(knots) - 1):
            if X_tst[0] >= knots[k] and X_tst[0] < knots[k + 1]:
                dtw = k
        first_stop_state = np.random.choice(np.array(list(state_dict.keys())), p = pi[dtw, :])
    
    else:
        first_stop_state = np.random.choice(np.array(list(state_dict.keys())), p = pi)


    future_states.append(first_stop_state)

    for i in range(len(X_tst) - 1):
        current_state = state_dict[future_states[-1]]

        if not time_homogeneous:

            for k in range(len(knots) - 1):
                if X_tst[i] >= knots[k] and X_tst[i] < knots[k + 1]:
                    dtw = k
            next_state = np.random.choice(np.array(list(state_dict.keys())), p = transition_matrix[dtw, current_state, :])
        
        elif not stop_homogeneous:
            
            next_state = np.random.choice(np.array(list(state_dict.keys())), p = transition_matrix[i, current_state, :])

        else:

            next_state = np.random.choice(np.array(list(state_dict.keys())), p = transition_matrix[current_state, :])

        future_states.append(next_state)

    return future_states

def pred_next_stop_delay(current_state, state_dict, transition_matrix, x_tst, knots, t, time_homogeneous = True, stop_homogeneous = True):

      if not time_homogeneous:

          for k in range(len(knots) - 1):
              if x_tst >= knots[k] and x_tst < knots[k + 1]:
                  dtw = k
          return np.random.choice(np.array(list(state_dict.keys())), p = transition_matrix[dtw, state_dict[current_state], :])
      
      elif not stop_homogeneous:
          
          return np.random.choice(np.array(list(state_dict.keys())), p = transition_matrix[t, state_dict[current_state], :])

      else:

          return np.random.choice(np.array(list(state_dict.keys())), p = transition_matrix[state_dict[current_state], :])

In [ ]:
#----- Parameters ----- 
slidewindow_len = 250 #Number of previous days to train on
time_homogeneous = True
stop_homogeneous = False
if not time_homogeneous and not stop_homogeneous:
    print('time inhomogeneous and stop inhomogeneous markov chains not implemented, keep time_homogeneous or stop_homogeneous True')
    exit() 
knots = [0, 21600, 32400, 50400, 64800, 93600] #DTWs time limits (for inhomogeneous chains)
#----------------------

#Init
state_dict = {'Early': 0, 'On Time': 1, 'Late': 2}
accu1 = 0
accu2 = 0

#Usefull constants
N, T = X_train.shape

for k in range(slidewindow_len, N):

    X_tr = X_train[k-slidewindow_len:k,:]
    y_tr = y_train[k-slidewindow_len:k,:]
    X_tst = X_train[k,:]
    y_tst = y_train[k,:]
    pi, transition_matrix = parametrization(state_dict, X_tr, y_tr, knots, time_homogeneous, stop_homogeneous)


    #Predict the next trip sequence of states
    future_states = pred_next_trip_delays(state_dict, pi, transition_matrix, X_tst, knots, time_homogeneous, stop_homogeneous)
    
    for t in range(T):

        if y_tst[t] == future_states[t]:
            accu1 += 1
    
    #Predict the state of the next stop
    if not time_homogeneous:

        for k in range(len(knots) - 1):
            if X_tst[0] >= knots[k] and X_tst[0] < knots[k + 1]:
                dtw = k
        first_state = np.random.choice(np.array(list(state_dict.keys())), p = pi[dtw, :]) #Prediction of the first stop state is random
    
    else:
        first_state = np.random.choice(np.array(list(state_dict.keys())), p = pi) #Prediction of the first stop state is random

    if y_tst[0] == first_state:
        accu2 += 1

    for t in range(0, T - 1):
        next_state = pred_next_stop_delay(y_tst[t], state_dict, transition_matrix, X_tst[t], knots, t, time_homogeneous, stop_homogeneous)
        
        if y_tst[t] == next_state:
            accu2 += 1


accu1 = accu1 /((N - slidewindow_len) * T)
accu2 = accu2 /((N - slidewindow_len) * T)
print(accu1)
print(accu2)